<a href="https://colab.research.google.com/github/mhmdtaha1999/DataClassification/blob/main/datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import sqrt
import numpy as np
from google.colab import drive
import os
import pandas as pd
import csv

In [ ]:
drive.mount('/content/drive')

trainFile = pd.read_csv('/content/drive/My Drive/train.csv')
testFile = pd.read_csv('/content/drive/My Drive/test.csv')

print(trainFile.columns)

trainFile.columns = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'label']

y_train = trainFile['label']


x_train = trainFile.drop('label', axis=1)


x_train.to_csv('/content/drive/My Drive/x_train.csv', index=False)
y_train.to_csv('/content/drive/My Drive/y_train.csv', index=False)

testFile.columns = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'label']
y_test = testFile['label']


x_test = testFile.drop('label', axis=1)


x_test.to_csv('/content/drive/My Drive/x_test.csv', index=False)
y_test.to_csv('/content/drive/My Drive/y_test.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['5.1', '3.5', '1.4', '0.2', '1'], dtype='object')


In [ ]:
import numpy as np

class KNN:
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for x in X_test:

            distances = [np.sqrt(np.sum((x - x_train)**2)) for x_train in self.X_train]

            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]

            counts = np.bincount(k_nearest_labels)
            label = np.argmax(counts)
            y_pred.append(label)
        return np.array(y_pred)

In [ ]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))


def initialize_centroids(data, k):
    indices = np.random.choice(len(data), k, replace=False)
    return data[indices]


def assign_clusters(data, centroids):
    clusters = [[] for _ in range(len(centroids))]
    for point in data:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        cluster_index = np.argmin(distances)
        clusters[cluster_index].append(point)
    return clusters


def update_centroids(clusters):
    centroids = [np.mean(cluster, axis=0) for cluster in clusters]
    return np.array(centroids)


def has_converged(old_centroids, centroids, tol=1e-4):
    return np.all(np.abs(np.array(centroids) - np.array(old_centroids)) < tol)

def kmeans(data, k, max_iter=100):
    centroids = initialize_centroids(data, k)
    for _ in range(max_iter):
        old_centroids = centroids.copy()
        clusters = assign_clusters(data, centroids)
        centroids = update_centroids(clusters)
        if has_converged(old_centroids, centroids):
            break
    return centroids, clusters


k = 3
centroids, clusters = kmeans(x_train.to_numpy(), k)


for i, centroid in enumerate(centroids):
    print(f"Cluster {i+1} centroid: {centroid}")
    print(f"Number of points in Cluster {i+1}: {len(clusters[i])}")

Cluster 1 centroid: [5.04411765 3.46764706 1.47941176 0.24411765]
Number of points in Cluster 1: 34
Cluster 2 centroid: [5.79761905 2.71666667 4.35238095 1.42619048]
Number of points in Cluster 2: 42
Cluster 3 centroid: [6.95       3.06538462 5.83076923 2.01153846]
Number of points in Cluster 3: 26


In [ ]:
def find_nearest_cluster(test_samples, cluster_centers):
    nearest_clusters = []
    for test_sample in test_samples:
        distances = [euclidean_distance(test_sample, center) for center in cluster_centers]
        nearest_cluster = np.argmin(distances)
        nearest_clusters.append(nearest_cluster)
    return nearest_clusters

cluster_centers = centroids
nearest_clusters = find_nearest_cluster(x_test.to_numpy(), cluster_centers)


for i, nearest_cluster in enumerate(nearest_clusters):
    print(f"Test sample {i+1} is closest to cluster {nearest_cluster+1}")



Test sample 1 is closest to cluster 1
Test sample 2 is closest to cluster 1
Test sample 3 is closest to cluster 1
Test sample 4 is closest to cluster 1
Test sample 5 is closest to cluster 1
Test sample 6 is closest to cluster 1
Test sample 7 is closest to cluster 1
Test sample 8 is closest to cluster 1
Test sample 9 is closest to cluster 1
Test sample 10 is closest to cluster 1
Test sample 11 is closest to cluster 1
Test sample 12 is closest to cluster 1
Test sample 13 is closest to cluster 1
Test sample 14 is closest to cluster 1
Test sample 15 is closest to cluster 2
Test sample 16 is closest to cluster 2
Test sample 17 is closest to cluster 2
Test sample 18 is closest to cluster 2
Test sample 19 is closest to cluster 2
Test sample 20 is closest to cluster 2
Test sample 21 is closest to cluster 2
Test sample 22 is closest to cluster 2
Test sample 23 is closest to cluster 2
Test sample 24 is closest to cluster 2
Test sample 25 is closest to cluster 2
Test sample 26 is closest to clust

In [ ]:
k = 3  # مقدار بهینه برای k

y_pred = []

for cluster_id in range(len(clusters)):
    cluster_data = np.array(clusters[cluster_id])
    cluster_indices = np.where(np.array(nearest_clusters) == cluster_id)[0]

    min_len = min(len(cluster_indices), len(cluster_data))
    cluster_indices = cluster_indices[:min_len]
    cluster_data = cluster_data[:min_len]

    cluster_labels = np.array(y_train)[cluster_indices]

    knn = KNN(k)
    knn.fit(cluster_data, cluster_labels)

    predicted_labels = knn.predict(x_test.to_numpy()[np.array(nearest_clusters) == cluster_id])

    y_pred.extend(predicted_labels)

print("Predicted labels:", y_pred)


Predicted labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.30434782608695654
Precision: 0.12911725955204217
Recall: 0.30434782608695654
F1-score: 0.18131359851988896


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
